In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
import geoplot.crs as ccrs
import geoplot as gplt
import shapely
%matplotlib inline

In [5]:
census_tracts = gpd.read_file("../data/boston_airbnb/Census2010_Tracts.shp")
census_tracts = census_tracts.to_crs(epsg=4326)
listings = pd.read_csv("../data/boston_airbnb/listings.csv")

In [10]:
airbnbs = listings[['latitude', 'longitude']].apply(lambda srs: Point(srs['longitude'], srs['latitude']), axis='columns')

In [11]:
listings = gpd.GeoDataFrame(data=listings, geometry=airbnbs)

In [39]:
# ax = gplt.kdeplot(listings, projection=ccrs.AlbersEqualArea(), shade=True, shade_lowest=False,
#                   clip=census_tracts.geometry)
# gplt.polyplot(census_tracts, projection=ccrs.AlbersEqualArea(), ax=ax, linewidth=0.5)

This is a case for `aggplot`! ...right, except I first need to fix `aggplot` up.